In [ ]:
import pygraphviz as pgv
import networkx as nx
import re
from normalizer import normalize_file, normalize_string
import os
import sys
current_dir = os.path.dirname(os.getcwd())
parent_dir = os.path.dirname(current_dir)
sys.path.append(current_dir)

from Model.ir_tokenizer import load_tokenizer


dot_file = "/home/damaoooo/Downloads/regraphv2/DataProcess/.movelim.dot"
instrumented = "/home/damaoooo/Downloads/regraphv2/DataProcess/ddg_exporter/build/instrumented.ll"
origin_ir = "/home/damaoooo/Downloads/regraphv2/DataProcess/ddg_exporter/test.ll"


def read_dot_file(dot_file):
    g = pgv.AGraph(dot_file)
    return g

def get_graph_numbers():
    g = read_dot_file(dot_file)
    for node in g.nodes():
        node_id = node.get_name()
        node_label = node.attr['label']
        # br i1 %.22.i7, label %noborder.exit17, label %'@2.i8', !my_id !57 (ID: 57)
        # Instruction number is the last part of the label
        if "!my_id" in node_label:
            instruction_number = re.findall(r'!my_id\s+!(\d+)', node_label)
            if instruction_number:
                instruction_number = instruction_number[0]
            else:
                instruction_number = "unknown"
            print(node_id, instruction_number)


from transformers import PreTrainedTokenizerFast


tokenizer_path = "/home/damaoooo/Downloads/regraphv2/DataProcess/output_tokenizer/llvm_ir_bpe.json"
tokenizer = load_tokenizer(tokenizer_path)

normalized_test = normalize_file(origin_ir)
normalized_instrumented = normalize_file(instrumented)

print(normalized_test)


Error normalizing file /home/damaoooo/Downloads/regraphv2/DataProcess/ddg_exporter/build/instrumented.ll: [Errno 2] No such file or directory: '/home/damaoooo/Downloads/regraphv2/DataProcess/ddg_exporter/build/instrumented.ll'
%var0 = type { i32 }

@func0 = external local_unnamed_addr global i8
@func1 = external local_unnamed_addr global ptr
@func2 = external global i64
@func3 = external thread_local global [65536 x i8]

declare ptr @func4(i64, i64, i64, i64, i64) local_unnamed_addr

declare i32 @func5(ptr) local_unnamed_addr

declare %var0 @func6(ptr, i32, i16, ptr) local_unnamed_addr

define i64 @func7(ptr %.1) local_unnamed_addr {
"@0":
%var1 = alloca ptr, align 8
%var2 = alloca [2 x i32], align 4
%var3 = alloca ptr, align 8
%var4 = alloca i16, align 2
store ptr %.1, ptr %var3, align 8
%.5.i = tail call ptr @llvm.threadlocal.address.p0(ptr @func3)
%.6.i = getelementptr i8, ptr %.5.i, i32 <MED_INT>
%.8.i = load i64, ptr %.6.i, align 4
store i16 0, ptr %var4, align 2
%.17.i = load i8,

<string>:9:44: warning: ptr type is only supported in -opaque-pointers mode
@conn = external local_unnamed_addr global ptr
                                           ^


In [56]:
g = read_dot_file(dot_file)

In [59]:
node_id_to_label = {}
tokenized = tokenizer(normalized_test, return_offsets_mapping=True)
offset_mapping = tokenized['offset_mapping']
input_ids = tokenized['input_ids']

assert len(input_ids) == len(offset_mapping), "Input IDs and offset mapping lengths do not match."

for node in g.nodes():
    node_id = node.get_name()
    node_label = node.attr['label']
    # print(node_label)
    # {"@0":\l|  %a1.i = alloca ptr, align 8\l  %value_list.i = alloca [2 x 
    # 我想要用提取{和\l 之间的内容
    match = re.search(r'\{(.*?)\\l', node_label, re.DOTALL)
    if match:
        content = match.group(1).strip()
        node_id_to_label[node_id] = content

label_to_token_position = {}

for node_id, label in node_id_to_label.items():
    # 这里可以对 label 进行进一步处理或分析
    
    start_idx = normalized_test.find(label)
    end_idx = start_idx + len(label)
    if start_idx == -1:
        print(f"Label '{label}' not found in normalized_test.")
        continue
    tokens = []
    for i, (start, end) in enumerate(offset_mapping):
        if start >= start_idx and end <= end_idx:
            tokens.append((i))
    label_to_token_position[label] = tokens

def extract_probability_from_edge(edge):
    """从边的tooltip属性中提取概率"""
    tooltip = edge.attr.get('tooltip', '')
    if 'Probability' in tooltip:
        # 使用正则表达式提取概率数值
        match = re.search(r'Probability\s+([\d.]+)%', tooltip)
        if match:
            return float(match.group(1))/100
    return None

cfg_edges = []
for edge in g.edges():
    source = edge[0]
    target = edge[1]
    probability = extract_probability_from_edge(edge)
    print(probability)
    source_label = node_id_to_label.get(source, "")
    target_label = node_id_to_label.get(target, "")
    
    if source_label in label_to_token_position and target_label in label_to_token_position:
        source_tokens = label_to_token_position[source_label]
        target_tokens = label_to_token_position[target_label]
    else:
        print(f"Edge from {source} to {target} has missing labels.")
        continue

    cfg_edges.append((source_tokens, target_tokens, probability))

# input_ids 转换为Token
input_tokens = tokenizer.convert_ids_to_tokens(input_ids)
print("Input Tokens:", input_tokens)

0.375
0.625
0.5
0.5
1.0
0.375
0.625
0.5
0.5
1.0
0.5
0.5
1.0
1.0
Input Tokens: ['<bos>', '%', 'var0', '=', 'type', '{', 'i32', '}', '@', 'func0', '=', 'external', 'local_unnamed_addr', 'global', 'i8', '@', 'func1', '=', 'external', 'local_unnamed_addr', 'global', 'ptr', '@', 'func2', '=', 'external', 'global', 'i64', '@', 'func3', '=', 'external', 'thread_local', 'global', '[', '65536', 'x', 'i8', ']', 'declare', 'ptr', '@', 'func4', '(', 'i64', ',', 'i64', ',', 'i64', ',', 'i64', ',', 'i64', ')', 'local_unnamed_addr', 'declare', 'i32', '@', 'func5', '(', 'ptr', ')', 'local_unnamed_addr', 'declare', '%', 'var0', '@', 'func6', '(', 'ptr', ',', 'i32', ',', 'i16', ',', 'ptr', ')', 'local_unnamed_addr', 'define', 'i64', '@', 'func7', '(', 'ptr', '%.', '1', ')', 'local_unnamed_addr', '{', '"@', '0', '":', '%', 'var1', '=', 'alloca', 'ptr', ',', 'align', '8', '%', 'var2', '=', 'alloca', '[', '2', 'x', 'i32', '],', 'align', '4', '%', 'var3', '=', 'alloca', 'ptr', ',', 'align', '8', '%', 'var4'

In [6]:
new_dot_path = "/home/damaoooo/Downloads/regraphv2/DataProcess/cfg_exporter/build/cfg_450a2bf4b3762de9b8037ea67284bb8a8176c79e7e457ee19fba9f404f94ba11.dot"
g = read_dot_file(new_dot_path)

In [11]:
for node in g.nodes():
    node_id = node.get_name()
    node_label = node.attr['label']
    print(f"Node ID: {node_id}, Label: {node_label.split(":")[0].replace("\\n", "")+":"}")

Node ID: Node0x6146ba2dd2d0, Label: "@0":
Node ID: Node0x6146ba2dd7e0, Label: "@2":
Node ID: Node0x6146ba2dd770, Label: "@3":
